In [ ]:
# Import Libraries
import pymysql
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
import matplotlib.pyplot as plt
import json

# Map nilai asesmen ke skor angka
nilai_mapping = {'BB': 1, 'MB': 2, 'BSH': 3, 'BSB': 4}

# Fungsi untuk parsing kolom 'hasil' JSON → skor rata-rata
def konversi_json_ke_skor(nilai_json):
    try:
        # Pastikan hanya string non-kosong
        if not isinstance(nilai_json, str) or nilai_json.strip() == "":
            return np.nan
        hasil_parsed = json.loads(nilai_json)
        skor = [nilai_mapping.get(v, np.nan) for v in hasil_parsed.values()]
        return np.nanmean(skor)
    except Exception:
        return np.nan

# Koneksi ke MySQL
def get_connection():
    return pymysql.connect(
        host='afl2ht.h.filess.io',
        port=61002,
        user='sistemtkdb_usefulheor',
        password='382f83f60b7560c62cad795c7e8b88ca8f9e8626',
        database='sistemtkdb_usefulheor'
    )

# Ambil Data dari MySQL
def get_data():
    connection = get_connection()
    query = '''
    SELECT 
        ia.id AS id_siswa,
        ia.nama_lengkap AS nama,
        ac.hasil,
        tk.id AS id_tumbuh_kembang,
        tk.tinggi_badan,
        tk.berat_badan,
        tk.lingkar_kepala,
        tk.umur,
        tk.tanggal_input,
        tk.created_at,
        tk.updated_at
    FROM 
        identitas_anak ia
    LEFT JOIN 
        hasil_asesmen_ceklis ac ON ia.id = ac.id_siswa
    LEFT JOIN 
        tumbuh_kembang tk ON ia.id = tk.id_siswa
    ORDER BY ia.id, tk.created_at
    '''
    data = pd.read_sql(query, connection)
    connection.close()
    return data

# Ambil data
data = get_data()

# Buat list semua id_siswa unik
siswa_ids = data['id_siswa'].unique()

# Simpan hasil prediksi
all_predictions = []

# Loop per siswa
for id_siswa in siswa_ids:
    df_siswa = data[data['id_siswa'] == id_siswa].copy()

    # Hanya proses jika data minimal 3 baris
    if df_siswa.shape[0] < 3:
        continue

    # Urutkan berdasarkan waktu input
    df_siswa = df_siswa.sort_values(by='created_at')

    # Tambahkan kolom hasil_skor (konversi dari JSON ke angka)
    df_siswa['hasil_skor'] = df_siswa['hasil'].apply(konversi_json_ke_skor)

    # Skip jika semua hasil kosong
    if df_siswa['hasil_skor'].isna().all():
        continue

    # Bagi data: latih = semua kecuali baris terakhir
    df_train = df_siswa.iloc[:-1]
    df_pred = df_siswa.iloc[-1:]

    # Drop kolom tidak relevan
    drop_cols = ['id_siswa', 'nama', 'hasil', 'hasil_skor', 'created_at', 'updated_at', 'tanggal_input']
    X_train = df_train.drop(columns=drop_cols, errors='ignore')
    X_pred = df_pred.drop(columns=drop_cols, errors='ignore')
    y_train = df_train['hasil_skor']

    # Isi NaN hanya untuk kolom numerik
    numeric_cols = X_train.select_dtypes(include=[np.number]).columns
    X_train[numeric_cols] = X_train[numeric_cols].fillna(X_train[numeric_cols].mean())
    X_pred[numeric_cols] = X_pred[numeric_cols].fillna(X_train[numeric_cols].mean())
    y_train = y_train.fillna(y_train.mean())

    # Buat fitur polinomial
    poly = PolynomialFeatures(degree=2)
    X_train_poly = poly.fit_transform(X_train)
    X_pred_poly = poly.transform(X_pred)

    # Train model
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train_poly, y_train)

    # Prediksi
    pred = model.predict(X_pred_poly)[0]

    # Simpan hasil prediksi
    all_predictions.append({
        'id_siswa': id_siswa,
        'nama': df_siswa['nama'].iloc[0],
        'hasil_asli': df_pred['hasil_skor'].values[0],
        'hasil_prediksi': pred
    })

# Buat dataframe hasil
hasil_df = pd.DataFrame(all_predictions)

# Tampilkan hasil
print("\nHasil prediksi per siswa:")
print(hasil_df)

# Visualisasi perbandingan
plt.figure(figsize=(10, 6))
plt.scatter(hasil_df['hasil_asli'], hasil_df['hasil_prediksi'], color='blue')
plt.plot([hasil_df['hasil_asli'].min(), hasil_df['hasil_asli'].max()],
         [hasil_df['hasil_asli'].min(), hasil_df['hasil_asli'].max()],
         color='red', linestyle='--')
plt.xlabel('Hasil Asli (Skor Rata-rata)')
plt.ylabel('Hasil Prediksi')
plt.title('Forecasting Hasil Asesmen per Siswa')
plt.grid(True)
plt.show()


OperationalError: (1045, "Access denied for user 'root'@'localhost' (using password: YES)")